# 🧠 Práctica Integrada de Clasificación  
**Curso:** Aprendizaje Supervisado (Clases 1–13)  
**Dataset:** `FraudeCanastas.csv`  

### 🎯 Objetivo
Aplicar el flujo completo de un problema de **clasificación supervisada** con los modelos vistos en clase.  

**Fases:**
1️⃣ Lectura del dataset  
2️⃣ EDA (Exploración de Datos)  
3️⃣ Preparación de Datos  
4️⃣ Elección de Modelo  
5️⃣ Entrenamiento y Prueba  
6️⃣ Evaluación del Modelo  

---

## 🧩 Paso 1. Lectura del Dataset

In [ ]:
import pandas as pd

# Ruta del archivo
DATA_PATH = '/mnt/data/FraudeCanastas.csv'

# Lectura y vista general
df = pd.read_csv(DATA_PATH)
print("Dimensiones del dataset:", df.shape)
df.head()

## 🔍 Paso 2. EDA (Exploración de Datos)

In [ ]:
# Información general
df.info()

# Estadísticas básicas
df.describe(include='all').T.head(10)

In [ ]:
# Distribución de clases
import seaborn as sns
import matplotlib.pyplot as plt

target_col = 'fraud_flag'
sns.countplot(x=target_col, data=df)
plt.title('Distribución de clases (fraude vs no fraude)')
plt.show()

# Revisión de nulos
df.isnull().sum().sort_values(ascending=False).head(10)

## 🧰 Paso 3. Preparación de Datos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

id_col = 'ID'
target_col = 'fraud_flag'

# Separar features y target
X = df.drop(columns=[target_col, id_col])
y = df[target_col]

# División en entrenamiento y prueba (estratificada)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Identificación de tipos de variables
cat_cols = [c for c in X_train.columns if X_train[c].dtype == 'object']
num_cols = [c for c in X_train.columns if X_train[c].dtype != 'object']

print(f"Variables categóricas: {len(cat_cols)} | Variables numéricas: {len(num_cols)}")

# Preprocesador
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', min_frequency=10), cat_cols),
    ('num', StandardScaler(), num_cols)
])


## ⚙️ Paso 4. Elección del Modelo


Selecciona **uno de los modelos vistos en clase** y justifica tu elección:

- **Perceptrón:** separador lineal (Clases 2–4).  
- **Árbol de Decisión:** reglas tipo *if–else*, basado en entropía o Gini (Clases 6–8).  
- **Regresión Logística:** clasificación probabilística con regularización (Clases 10–13).  


In [ ]:
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# Selección del modelo
modelo = 'logistica'  # opciones: 'perceptron', 'arbol', 'logistica'

if modelo == 'perceptron':
    clf = Perceptron(class_weight='balanced', max_iter=1000, random_state=42)
elif modelo == 'arbol':
    clf = DecisionTreeClassifier(criterion='entropy', class_weight='balanced',
                                 min_samples_leaf=50, random_state=42)
else:
    clf = LogisticRegression(class_weight='balanced', max_iter=3000, solver='liblinear', random_state=42)

pipe = Pipeline([('prep', preprocessor), ('model', clf)])
pipe

## 🧮 Paso 5. Entrenamiento y Prueba

In [ ]:
# Entrenamiento
pipe.fit(X_train, y_train)

# Predicciones
y_pred = pipe.predict(X_test)

# Probabilidades (si aplica)
if hasattr(pipe, 'predict_proba'):
    y_proba = pipe.predict_proba(X_test)[:, 1]
else:
    y_proba = None

print("Entrenamiento y predicción completados.")

## 📊 Paso 6. Evaluación del Modelo

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_auc_score, roc_curve

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot(cmap='Blues')
plt.title('Matriz de Confusión')
plt.show()

# Métricas principales
print(classification_report(y_test, y_pred))

# ROC-AUC
if y_proba is not None:
    auc = roc_auc_score(y_test, y_proba)
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.plot(fpr, tpr, label=f'AUC = {auc:.3f}')
    plt.plot([0,1],[0,1],'--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.title('Curva ROC')
    plt.show()
else:
    print("El modelo no genera probabilidades.")

## 🧾 Conclusión Final


- ¿Qué modelo elegiste y por qué?  
- ¿Qué métricas fueron las más relevantes?  
- ¿Cuál fue el principal tipo de error (FP/FN)?  
- ¿Cómo podrías mejorar el desempeño del modelo?  
